In [1]:
## Use data of a production machine, where each row contains informations about a particular part
## Ceate a data point
## In this case, instead of "asperity" we will have a binary attribute "Broken" to say if the part is broken with aspirity>1 or not with asperity<1, 
dp1 = {'partno': 100, 'maxtemp': 35, 'mintemp': 35, 'maxvibration': 12, 'Broken': 0}
dp2 = {'partno': 101, 'maxtemp': 46, 'mintemp': 35, 'maxvibration': 21, 'Broken': 0}
dp3 = {'partno': 130, 'maxtemp': 56, 'mintemp': 46, 'maxvibration': 3412, 'Broken': 1}
dp4 = {'partno': 131, 'maxtemp': 58, 'mintemp': 48, 'maxvibration': 3542, 'Broken': 1}

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200409151707-0000
KERNEL_ID = 3be29ba1-b754-4ddc-be6d-6d4088db23ad


In [3]:
## We will change this regression data set into a binary classification data set. Then our rule gets more simple and also more precise.
def predict(dp):
    if dp['maxvibration']>100:
        return 1
    else:
        return 0

In [4]:
predict(dp1)

0

In [13]:
## changed our linear regression model to an regression one.
## In order to achieve this, just add a sigmoid computation step to our site
import math

def sigmoid(x):
    return 1/(1+math.exp(-x))

In [14]:
## Logistic regression is to create a binary relationship between filds by using fild numbers and a rate 'W' 
w1 = 0.33
w2 = 0
w3 = 0
w4 = 13/3412

def mlpredict(dp):
    return 0 if sigmoid(w1+w2*dp['maxtemp']+w3*dp['mintemp']+w4*dp['maxvibration'])<0.7 else 1

In [15]:
mlpredict(dp1)

0

In [16]:
# The code was removed by Watson Studio for sharing.

[Row(x=22, y=49, z=35, class='Brush_teeth', source='Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt'),
 Row(x=22, y=49, z=35, class='Brush_teeth', source='Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt'),
 Row(x=22, y=52, z=35, class='Brush_teeth', source='Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt'),
 Row(x=22, y=52, z=35, class='Brush_teeth', source='Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt'),
 Row(x=21, y=52, z=34, class='Brush_teeth', source='Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt')]

In [17]:
df = df_data_1

In [41]:
## We split the dataset to 0.8 train data and 0.2 test data
splits = df.randomSplit([0.8,0.2])
df_train = splits[0]
df_test = splits[1]

In [29]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Normalizer
from pyspark.ml.linalg import Vectors

indexer = StringIndexer(inputCol='class', outputCol='label')
assembler = VectorAssembler(inputCols=['x','y','z'], outputCol='features')
normalizer = Normalizer(inputCol='features', outputCol='features_norm')

In [30]:
from pyspark.ml.classification import LogisticRegression

In [31]:
Lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [42]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[indexer,assembler,normalizer,Lr])
model = pipeline.fit(df_train)


In [43]:
## Compute predictions
prediction = model.transform(df_train)

In [44]:
prediction.show()

+---+---+---+--------------+--------------------+-----+---------------+--------------------+--------------------+--------------------+----------+
|  x|  y|  z|         class|              source|label|       features|       features_norm|       rawPrediction|         probability|prediction|
+---+---+---+--------------+--------------------+-----+---------------+--------------------+--------------------+--------------------+----------+
|  0| 10| 28|     Getup_bed|Accelerometer-201...|  0.0|[0.0,10.0,28.0]|[0.0,0.3363363969...|[0.64772850969516...|[0.12888508466137...|       0.0|
|  0| 11| 38| Sitdown_chair|Accelerometer-201...|  7.0|[0.0,11.0,38.0]|[0.0,0.2780580765...|[0.64772850969516...|[0.12888508466137...|       0.0|
|  0| 12| 39| Sitdown_chair|Accelerometer-201...|  7.0|[0.0,12.0,39.0]|[0.0,0.2940858488...|[0.64772850969516...|[0.12888508466137...|       0.0|
|  0| 16| 31|     Getup_bed|Accelerometer-201...|  0.0|[0.0,16.0,31.0]|[0.0,0.4586429197...|[0.64772850969516...|[0.12888508

In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [46]:
eval = MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol('prediction')

In [47]:
eval.evaluate(prediction)

0.12888490693154522

In [49]:
## So now let's do the test using the test data. So basically, we do the same. So we feed the df_test
model = pipeline.fit(df_test)
## And now we compute our predictions using the model.transform(df_test).
predictionTest = model.transform(df_test)

##And we can re-use our evaluator. And we hopefully should see a similar result.
eval.evaluate(predictionTest)

## Okay, cool, it's nearly 13% as well. And that's actually cool, if we get nearly the same number but a slightly lower number, 
## that means everything is all right.



0.130867267640659